In [1]:
print("Activating ipynb")
print ("Importing libs")
import os 
import numpy as np
import pandas as pd

Activating ipynb
Importing libs


In [2]:
# Global
years = [x for x in range (2012, 2022)]

In [3]:
# Load Initial Data
print("Loading Initial Game Data")
games_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/"
games_raw_2012 = pd.read_csv(games_folder_path + '2012.csv', index_col=False, parse_dates=['date'])
games_raw_2013 = pd.read_csv(games_folder_path + '2013.csv', index_col=False, parse_dates=['date'])
games_raw_2014 = pd.read_csv(games_folder_path + '2014.csv', index_col=False, parse_dates=['date'])
games_raw_2015 = pd.read_csv(games_folder_path + '2015.csv', index_col=False, parse_dates=['date'])
games_raw_2016 = pd.read_csv(games_folder_path + '2016.csv', index_col=False, parse_dates=['date'])
games_raw_2017 = pd.read_csv(games_folder_path + '2017.csv', index_col=False, parse_dates=['date'])
games_raw_2018 = pd.read_csv(games_folder_path + '2018.csv', index_col=False, parse_dates=['date'])
games_raw_2019 = pd.read_csv(games_folder_path + '2019.csv', index_col=False, parse_dates=['date'])
games_raw_2020 = pd.read_csv(games_folder_path + '2020.csv', index_col=False, parse_dates=['date'])
games_raw_2021 = pd.read_csv(games_folder_path + '2021.csv', index_col=False, parse_dates=['date'])

all_games = os.path.join("C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv")
all_games_raw = pd.read_csv(all_games, index_col=False, parse_dates=['date'])

Loading Initial Game Data


In [4]:
# Load Stats
stats_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Players/"
stats_raw_2012 = pd.read_csv(stats_folder_path + '2012.csv', index_col=False)
stats_raw_2013 = pd.read_csv(stats_folder_path + '2013.csv', index_col=False)
stats_raw_2014 = pd.read_csv(stats_folder_path + '2014.csv', index_col=False)
stats_raw_2015 = pd.read_csv(stats_folder_path + '2015.csv', index_col=False)
stats_raw_2016 = pd.read_csv(stats_folder_path + '2016.csv', index_col=False)
stats_raw_2017 = pd.read_csv(stats_folder_path + '2017.csv', index_col=False)
stats_raw_2018 = pd.read_csv(stats_folder_path + '2018.csv', index_col=False)
stats_raw_2019 = pd.read_csv(stats_folder_path + '2019.csv', index_col=False)
stats_raw_2020 = pd.read_csv(stats_folder_path + '2020.csv', index_col=False)
stats_raw_2021 = pd.read_csv(stats_folder_path + '2021.csv', index_col=False)

all_stats = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Players/stats_sorted.csv"
all_stats_raw = pd.read_csv(all_stats, index_col=False)

C:\Users\Craig\AppData\Local\Temp\ipykernel_2500\4262665646.py:15: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  all_stats_raw = pd.read_csv(all_stats, index_col=False)


## Build Y value 
Result of game based on Home Team (Win = 2, Draw = 1, Lose = 0)

In [5]:
def homeWin(row):
    homeScore = row["homeTeamScore"]
    awayScore = row["awayTeamScore"]
    row['homeWin'] = 2 if homeScore > awayScore else 0 if homeScore < awayScore else 1 
        
    return row
  
results_2012 = games_raw_2012.apply(homeWin, axis=1)
y_true_2012 = results_2012['homeWin']
results_2013 = games_raw_2013.apply(homeWin, axis=1)
y_true_2013 = results_2013['homeWin']
results_2014 = games_raw_2014.apply(homeWin, axis=1)
y_true_2014 = results_2014['homeWin']
results_2015 = games_raw_2015.apply(homeWin, axis=1)
y_true_2015 = results_2015['homeWin']
results_2016 = games_raw_2016.apply(homeWin, axis=1)
y_true_2016 = results_2016['homeWin']
results_2017 = games_raw_2017.apply(homeWin, axis=1)
y_true_2017 = results_2017['homeWin']
results_2018 = games_raw_2018.apply(homeWin, axis=1)
y_true_2018 = results_2018['homeWin']
results_2019 = games_raw_2019.apply(homeWin, axis=1)
y_true_2019 = results_2019['homeWin']
results_2020 = games_raw_2020.apply(homeWin, axis=1)
y_true_2020 = results_2020['homeWin']
results_2021 = games_raw_2021.apply(homeWin, axis=1)
y_true_2021 = results_2021['homeWin']
all_results = all_games_raw.apply(homeWin, axis=1)
y_true_all = all_results['homeWin']

In [18]:
# results_clean_2012 = results_2012
# results_clean_2012.columns
# results_clean_2012 = results_clean_2012.drop(['year', 'startTime', 'attendance'], axis=1)
# results_clean_2012.columns

a_2012 = results_2012
a_2013 = results_2013
a_2014 = results_2014
a_2015 = results_2015
a_2016 = results_2016
a_2017 = results_2017
a_2018 = results_2018
a_2019 = results_2019
a_2020 = results_2020
a_2021 = results_2021

## Add Previous Game Result
Set default dict to store results for each team
Only a boolean value, draws only account for ~1% of all games 
Default to false
If the last game was a win set to true
# TODO 
add draws in and see if there is a difference

In [20]:
# a_2012["homeTeamLastWin"] = False
# a_2012["awayTeamLastWin"] = False

from collections import defaultdict
won_last = defaultdict(bool)

def teamLastWin(row):
    row["homeTeamLastWin"] = False
    row["awayTeamLastWin"] = False
    home_team = row["homeTeam"]
    away_team = row["awayTeam"]
    row["homeTeamLastWin"] = won_last[home_team]
    row["awayTeamLastWin"] = won_last[away_team]
    
    won_last[home_team] = row["homeWin"] == 2
    won_last[away_team] = row["homeWin"] == 0
    
    return row
  
a_2013 = a_2013.apply(teamLastWin, axis=1)
a_2014 = a_2014.apply(teamLastWin, axis=1)
a_2015 = a_2015.apply(teamLastWin, axis=1)
a_2016 = a_2016.apply(teamLastWin, axis=1)
a_2017 = a_2017.apply(teamLastWin, axis=1)
a_2018 = a_2018.apply(teamLastWin, axis=1)
a_2019 = a_2019.apply(teamLastWin, axis=1)
a_2020 = a_2020.apply(teamLastWin, axis=1)
a_2021 = a_2021.apply(teamLastWin, axis=1)

## Computed Variables
These variables did not appear in the original data set but were added later

Travel Data: Used to determine the impact of travel on teams
             Determined by comparing the venue of the upcoming match to the teams home location

Break: Finds the break between a teams last and next game

Team Changes: The number of players that are different in the from the previous match
              The number of games lost/gained from these changes
              The expected score lost/gained from the incoming players
              # TODO split games out and games in into separate variables

Team Composition: The average/total score of all the selected players
                  Done this way to reduce number of features
                  
Previous 5 games result: Totals each teams previous games results

             

In [22]:
## Travel Distance
import travel_distance as td
def travel(row):
    row['awayTravel'] = td.team_travel(row['awayTeam'], row['venue'])
    row['homeTravel'] = td.team_travel(row['homeTeam'], row['venue'])
    return row
# a_2012 = a_2012.apply(travel, axis=1)
a_2013 = a_2013.apply(travel, axis=1)
a_2012 = a_2012.apply(travel, axis=1)
a_2013 = a_2013.apply(travel, axis=1)
a_2014 = a_2014.apply(travel, axis=1)
a_2015 = a_2015.apply(travel, axis=1)
a_2016 = a_2016.apply(travel, axis=1)
a_2017 = a_2017.apply(travel, axis=1)
a_2018 = a_2018.apply(travel, axis=1)
a_2019 = a_2019.apply(travel, axis=1)
a_2020 = a_2020.apply(travel, axis=1)
a_2021 = a_2021.apply(travel, axis=1)



## Team Similarity
This will add the number of changes that were made week to week as well as the loss/gain of games experience resulting from the change

TODO
Add player ratings loss/gain when implemented

In [23]:
import team_changes as tc

def getPreviousGame(round, team, date, year):
    if year == 2012: l = a_2012.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2013: l = a_2013.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2014: l = a_2014.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2015: l = a_2015.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2016: l = a_2016.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2017: l = a_2017.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2018: l = a_2018.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2019: l = a_2019.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2020: l = a_2020.query('date < @date and (homeTeam == @team or awayTeam == @team)')
    elif year == 2021: l = a_2021.query('date < @date and (homeTeam == @team or awayTeam == @team)')

    return l.values[-1][0]
    
def getChanges(row):
    row['homeTeamChanges'] = 0
    row['awayTeamChanges'] = 0
    row['homeGameDifference'] = 0
    row['awayGameDifference'] = 0
    row['homeTeamImportanceDifference'] = 0
    row['awayTeamImportanceDifference'] = 0

    if row['round'] == '1':
        return row
    homeTeam = row['homeTeam']
    awayTeam = row['awayTeam']
    round = row['round']
    gameId = row['gameId']
    year = row['year']

    prevGameHomeTeam = getPreviousGame(round, homeTeam, row['date'], int(year))
    prevGameAwayTeam = getPreviousGame(round, awayTeam, row['date'], int(year))
    homeChanges = tc.team_similarity(gameId, prevGameHomeTeam, homeTeam)
    awayChanges = tc.team_similarity(gameId, prevGameAwayTeam, awayTeam)
    
    row[['homeTeamChanges','awayTeamChanges']] = homeChanges[0],awayChanges[0]
    row[['homeGameDifference','awayGameDifference']] = homeChanges[1],awayChanges[1]
    row[['homeTeamImportanceDifference', 'awayTeamImportanceDifference']] = homeChanges[2], awayChanges[2]
    return row

# a_2012 = a_2012.apply(getChanges, axis = 1)
a_2013 = a_2013.apply(getChanges, axis = 1)
a_2014 = a_2014.apply(getChanges, axis = 1)
a_2015 = a_2015.apply(getChanges, axis = 1)
a_2016 = a_2016.apply(getChanges, axis = 1)
a_2017 = a_2017.apply(getChanges, axis = 1)
a_2018 = a_2018.apply(getChanges, axis = 1)
a_2019 = a_2019.apply(getChanges, axis = 1)
a_2020 = a_2020.apply(getChanges, axis = 1)
a_2021 = a_2021.apply(getChanges, axis = 1)

-57.02777777777778
-43.16666666666667
-78.69097222222223
-17.5
-49.5
-66.66071428571429
-80.75
-49.77777777777778
-69.5
-68.44097222222223
-76.88194444444444
-47.83333333333333
-91.67063492063491
-17.55
-51.617063492063494
-91.33333333333333
-24.22727272727272
-0.875
-5.422222222222217
24.75
-1.4715873015872987
5.750992063492063
8.125
-26.095568783068785
-10.202083333333348
-19.65814393939394
29.510317460317452
16.34848484848485
53.625
-1.2708333333333286
44.75892857142857
13.429754273504287
26.25
-3.5
19.45947802197803
2.727083333333333
23.239947552447553
34.50446428571428
-18.411904761904758
10.792048229548229
0.2066137566137627
-22.636309523809523
-25.29273504273504
-17.1939935064935
-13.760416666666657
-25.828571428571443
25.95611772486773
2.602331002331013
10.726587301587308
15.979166666666668
-22.147863247863246
10.824074074074076
18.535714285714292
9.668373015873023
-25.82440476190476
-27.364177489177486
21.811249999999994
-19.473070818070816
-19.79999999999999
-19.9615384615384

In [10]:
a_2012_test = a_2012.head(18)

In [25]:
# Player Importance
import player_rankings as pr
   
def getTeamImportance(row):
    row['homeTeamImportanceLastGame'] = 0
    row['awayTeamImportanceLastGame'] = 0
    row['homeTeamImportanceLastFiveGames'] = 0
    row['awayTeamImportanceLastFiveGames'] = 0
    row['homeTeamImportanceSeasonAverage'] = 0
    row['awayTeamImportanceSeasonAverage'] = 0

    if row['round'] == '1':
        return row
    homeTeam = row['homeTeam']
    awayTeam = row['awayTeam']
    gameId = row['gameId']

    homeImportance = pr.calculate_team_previous_game(gameId, homeTeam)
    print(homeImportance)
    awayImportance = pr.calculate_team_previous_game(gameId, awayTeam)
    homeImportancePrevFive = pr.calculate_team_previous_five(gameId, homeTeam)
    awayImportancePrevFive = pr.calculate_team_previous_five(gameId, awayTeam)
    homeImportanceAvg = pr.calculate_team_player_average(gameId, homeTeam)
    awayImportanceAvg = pr.calculate_team_player_average(gameId, awayTeam)
    
    row[['homeTeamImportanceLastGame','awayTeamImportanceLastGame']] = homeImportance, awayImportance
    row[['homeTeamImportanceLastFiveGames','awayTeamImportanceLastFiveGames']] = homeImportancePrevFive[1],awayImportancePrevFive[1]
    row[['homeTeamImportanceSeasonAverage', 'awayTeamImportanceSeasonAverage']] = homeImportanceAvg[1],awayImportanceAvg[1] 
    return row

# a_2012 = a_2012.apply(getTeamImportance, axis = 1)
a_2013 = a_2013.apply(getTeamImportance, axis = 1)
a_2014 = a_2014.apply(getTeamImportance, axis = 1)
a_2015 = a_2015.apply(getTeamImportance, axis = 1)
a_2016 = a_2016.apply(getTeamImportance, axis = 1)
a_2017 = a_2017.apply(getTeamImportance, axis = 1)
a_2018 = a_2018.apply(getTeamImportance, axis = 1)
a_2019 = a_2019.apply(getTeamImportance, axis = 1)
a_2020 = a_2020.apply(getTeamImportance, axis = 1)
a_2021 = a_2021.apply(getTeamImportance, axis = 1)

67.4090909090909
71.88636363636364
75.22727272727273
70.54545454545455
68.79545454545455
65.13636363636364
70.22727272727273
83.97727272727273
67.02272727272727
68.56818181818181
73.88636363636364
61.86363636363637
71.0
69.22727272727273
69.04545454545455
75.18181818181819
57.04545454545455
69.36363636363636
69.31818181818181
68.25
78.54545454545455
81.72727272727273
65.54545454545455
70.52272727272727
62.06818181818182
70.68181818181819
73.06818181818181
63.52272727272727
79.1590909090909
66.8409090909091
67.1590909090909
65.25
67.4090909090909
73.02272727272727
62.77272727272727
74.75
69.0
66.88636363636364
76.06818181818181
68.6590909090909
75.22727272727273
77.93181818181819
71.31818181818181
65.27272727272727
65.75
68.20454545454545
74.81818181818181
67.93181818181819
75.02272727272727
65.47727272727273
74.5909090909091
71.77272727272727
56.13636363636363
72.8409090909091
65.88636363636364
69.8409090909091
76.38636363636364
77.63636363636364
68.95454545454545
70.31818181818181
59.

In [24]:
# a_2012.to_csv(games_folder_path + '2012_with_avgs.csv', index=False)
a_2013.to_csv(games_folder_path + '2013_with_avgs.csv', index=False)
a_2014.to_csv(games_folder_path + '2014_with_avgs.csv', index=False)
a_2015.to_csv(games_folder_path + '2015_with_avgs.csv', index=False)
a_2016.to_csv(games_folder_path + '2016_with_avgs.csv', index=False)
a_2017.to_csv(games_folder_path + '2017_with_avgs.csv', index=False)
a_2018.to_csv(games_folder_path + '2018_with_avgs.csv', index=False)
a_2019.to_csv(games_folder_path + '2019_with_avgs.csv', index=False)
a_2020.to_csv(games_folder_path + '2020_with_avgs.csv', index=False)
a_2021.to_csv(games_folder_path + '2021_with_avgs.csv', index=False)

## One Hot Encoding
The tree models used for feature engineering cannot take in string inputs
Value encoding can be used to change teams to int values, but then the model will read this as distance i.e. the team with value 1 will be read as being closer to the team with value 2 than the team with value 18. 
One Hot Encoding creates a bool variable for each team which is 1 when the team is playing.

# TODO
See if the model performs better with the simple label encoding. 
One hot encoding creates an issue with decision trees as it creates lots of "empty" splits, there will now be 34 variables which the model will split on 0, which can skew the model.

In [ ]:
# One Hot Encoding Teams
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(all_results["homeTeam"].values)

home_teams_2012 = encoding.transform(a_2012["homeTeam"].values)
away_teams_2012 = encoding.transform(a_2012["awayTeam"].values)

all_teams_2012 = np.vstack([home_teams_2012, away_teams_2012]).T
all_teams_2012[:5], all_teams_2012.shape

oneHot = OneHotEncoder()
X_teams_2012 = oneHot.fit_transform(all_teams_2012).todense()
X_teams_2012.shape

## One Hot Encoding Venues
Same issues as above, but simplified slightly as there are only 23 stadiums

In [ ]:
# One Hot Encoding Venues
encoding = LabelEncoder()
encoding.fit(all_results["venue"].values)
venues = encoding.transform(a_2012['venue'].values)

all_venues = np.vstack(venues).T
venues = venues.reshape(-1,1)
venues[:5], venues.shape

oneHot = OneHotEncoder()
X_venues = oneHot.fit_transform(venues).todense()
X_venues = np.asarray(X_venues)
X_venues.shape